<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [ ]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [ ]:
# a)

df_01 = pd.read_csv(archivos_anio[0])
df_02 = pd.read_csv(archivos_anio[1])

df_01.columns = df_01.columns.str.lower()
df_02.columns = df_02.columns.str.lower() # Aqui hacemos que todas las columnas ahora solo esten en minusculas

df_anio = pd.concat([df_01, df_02], ignore_index=True)
df_anio.head(10)

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN
5,ARG,2001,12.0,8.0
6,ARM,2001,NaN,NaN
7,ATG,2001,NaN,NaN
8,AUS,2001,3.5,48.0
9,AUT,2001,7.5,86.0


In [ ]:
# b)
ISO=df_codigos.groupby('codigo_iso') #agrupamos la tabla por codigos
numpaisesxiso=ISO['pais'].nunique() #contamos la cantidad de paises distintos que aparecen en cada codigo
codigos_duplicados=numpaisesxiso[numpaisesxiso>1].index.to_list() #creamos una tabla con los codigos que tienen mas de 1 pais asociado
df_codigos.loc[df_codigos['codigo_iso'].isin(codigos_duplicados)] #buscamos en el df de codigos donde aparece el codigo duplicado para elegir cual borrar

,codigo_iso,pais
179,ZWE,Zimbabue
180,ZWE,malo


In [ ]:
#notamos que el codigo que esta malo es el de la fila 180
df_codigos=df_codigos.drop(180) #eliminamos la fila

In [ ]:
#c)
df=pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner') #combinamos los dataframes manteniendo solo las filas que coinciden los codigos ISO
df.head(10)

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos
5,ARG,2001,12.0,8.0,Argentina
6,ARM,2001,NaN,NaN,Armenia
7,ATG,2001,NaN,NaN,Antigua y Barbuda
8,AUS,2001,3.5,48.0,Australia
9,AUT,2001,7.5,86.0,Austria




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [ ]:
# FIXME
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB


In [ ]:
#asi notamos que df tiene 3060 filas y 5 columnas
#las columnas se llaman codigo_iso, anio, indice, ranking y pais
#las columas codigo_iso y pais tienen datos tipo object, la columna anio tiene datos tipo int64 y las columnas indice y ranking tienen datos tipo float 64
#bajo mi analisis no veo que hayan datos inesperados en la tabla ya que cada tipo de dato es consistente con lo esperado segun la columna

In [ ]:
df.describe()

,anio,indice,ranking
count,3060.000000,2664.000000,2837.000000
mean,2009.941176,205.782316,477.930913
std,5.786024,2695.525264,6474.935347
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.295000,34.000000
50%,2009.000000,28.000000,70.000000
75%,2015.000000,41.227500,110.000000
max,2019.000000,64536.000000,121056.000000


In [ ]:
#no entiendo que tengo que observar acerca del indice y ranking que me piden, puedo ver que tienen una cantidad de datos distintas y que el ranking es mayor al indice pero no se si esto es util

In [ ]:
max_indice = df['indice'].max()
min_indice = df['indice'].min()
mean_indice = df['indice'].mean()

print("Máximo de la columna 'indice':", max_indice)
print("Mínimo de la columna 'indice':", min_indice)
print("Promedio de la columna 'indice':", mean_indice)

Máximo de la columna 'indice': 64536.0
Mínimo de la columna 'indice': 0.0
Promedio de la columna 'indice': 205.7823160660661


In [ ]:
max_rank=df['ranking'].max()
min_rank=df['ranking'].min()

print("\nPaís(es) con el valor máximo de indice:")
display(df[df['indice'] == max_indice])

print("\nPaís(es) con el valor mínimo de indice:")
display(df[df['indice'] == min_indice])

print("\nPaís(es) con el valor máximo de ranking:")
display(df[df['ranking'] == max_rank])

print("\nPaís(es) con el valor mínimo de ranking:")
display(df[df['ranking'] == min_rank])


País(es) con el valor máximo de indice:


,codigo_iso,anio,indice,ranking,pais
2069,KSV,2014,64536.0,120614.0,Kosovo



País(es) con el valor mínimo de indice:


,codigo_iso,anio,indice,ranking,pais
1304,DNK,2008,0.0,2.0,Dinamarca
1313,FIN,2008,0.0,2.0,Finlandia
1335,IRL,2008,0.0,2.0,Irlanda
1382,NOR,2008,0.0,2.0,Noruega
1412,SWE,2008,0.0,2.0,Suecia
1468,CHE,2009,0.0,2.0,Suiza
1493,FIN,2009,0.0,2.0,Finlandia
1518,ISL,2009,0.0,2.0,Islandia
1561,NLD,2009,0.0,2.0,Países Bajos
1562,NOR,2009,0.0,2.0,Noruega



País(es) con el valor máximo de ranking:


,codigo_iso,anio,indice,ranking,pais
2249,KSV,2015,64527.0,121056.0,Kosovo



País(es) con el valor mínimo de ranking:


,codigo_iso,anio,indice,ranking,pais
53,FIN,2001,0.50,1.0,Finlandia
78,ISL,2001,0.50,1.0,Islandia
121,NLD,2001,0.50,1.0,Países Bajos
122,NOR,2001,0.50,1.0,Noruega
233,FIN,2002,0.50,1.0,Finlandia
258,ISL,2002,0.50,1.0,Islandia
301,NLD,2002,0.50,1.0,Países Bajos
302,NOR,2002,0.50,1.0,Noruega
388,CHE,2003,0.50,1.0,Suiza
404,DNK,2003,0.50,1.0,Dinamarca


In [ ]:
# Contamos los valores nulos por columna
nulos = df.isnull().sum()
proporcion_nulos = nulos / len(df)

print("Valores nulos por columna:")
print(nulos)

print("\nProporción de valores nulos por columna:")
print(proporcion_nulos)
filas_duplicadas=df.duplicated().sum()
print("\nCantidad de filas duplicadas:", filas_duplicadas)

Valores nulos por columna:
codigo_iso      0
anio            0
indice        396
ranking       223
pais            0
dtype: int64

Proporción de valores nulos por columna:
codigo_iso    0.000000
anio          0.000000
indice        0.129412
ranking       0.072876
pais          0.000000
dtype: float64

Cantidad de filas duplicadas: 0


In [ ]:
#podemos notar que no hay columnas con mas del 30% de datos faltantes

In [ ]:
paises_distintos = df['pais'].nunique()
print("Cantidad de países distintos:", paises_distintos)
cantidad_años=df['anio'].nunique()
print("Cantidad de años distintos:", cantidad_años)
iso_distintos=df['codigo_iso'].nunique()
print("Cantidad de códigos ISO distintos:", iso_distintos)

Cantidad de países distintos: 179
Cantidad de años distintos: 17
Cantidad de códigos ISO distintos: 180


In [ ]:
#podemos observar que hay 1 iso mas que paises, por lo que vamos a aplicar nuevamente lo hecho en 1 para encontrar la inconsistencia

In [ ]:
grupopais=df.groupby('pais') #agrupamos la tabla por paises
numisosxpais=grupopais['codigo_iso'].nunique() #contamos la cantidad de paises distintos que aparecen en cada codigo
codigos_duplicados=numisosxpais[numisosxpais>1].index.to_list() #creamos una tabla con los codigos que tienen mas de 1 pais asociado
df.loc[df['pais'].isin(codigos_duplicados)] #buscamos en el df de codigos donde aparece el codigo duplicado para elegir cual borrar

,codigo_iso,anio,indice,ranking,pais
118,NER,2001,18.50,17.0,Nigeria
119,NGA,2001,15.50,14.0,Nigeria
298,NER,2002,15.75,16.0,Nigeria
299,NGA,2002,31.50,55.0,Nigeria
478,NER,2003,18.33,26.0,Nigeria
479,NGA,2003,37.75,67.0,Nigeria
658,NER,2004,13.00,13.0,Nigeria
659,NGA,2004,38.75,61.0,Nigeria
838,NER,2005,24.50,39.0,Nigeria
839,NGA,2005,32.23,56.0,Nigeria


In [ ]:
#aqui podemos notar que nigeria tiene 2 codigos Iso asociados, lo que es una inconsistencia

['Nigeria']




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [ ]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america =  df.loc[df['codigo_iso'].isin(america)] # FIX ME
for año in df_america['anio'].unique(): #Recorremos los años de manera unica (no pasamos 2 veces por el mismo año)
    df_año = df_america[df_america['anio'] == año] #creamos un df por cada año para que el analisis sea mas sencillo
    if df_año['indice'].isnull().sum()!= df_año.shape[0]: #verificamos que haya informacion de cada año
        indice_minimo = df_año['indice'].max()
        indice_maximo = df_año['indice'].min()
        maximo = df_año[df_año['indice']== indice_maximo]
        minimo = df_año[df_año['indice']== indice_minimo] #extraemos la informacion
        print("El año", año, "el pais con mejor indice fue")
        display(maximo)
        print("El año", año, "el pais con peor indice fue")
        display(minimo)
    else:
      print("no hay informacion para el año",año)




El año 2001 el pais con mejor indice fue


,codigo_iso,anio,indice,ranking,pais
27,CAN,2001,0.8,2.0,Canadá


El año 2001 el pais con peor indice fue


,codigo_iso,anio,indice,ranking,pais
39,CUB,2001,90.3,99.0,Cuba


El año 2002 el pais con mejor indice fue


...

,codigo_iso,anio,indice,ranking,pais
2739,CUB,2018,68.9,172.0,Cuba


El año 2019 el pais con mejor indice fue


,codigo_iso,anio,indice,ranking,pais
2961,JAM,2019,11.13,8.0,Jamaica


El año 2019 el pais con peor indice fue


,codigo_iso,anio,indice,ranking,pais
2919,CUB,2019,63.81,169.0,Cuba


In [ ]:
maximo=df_america[df_america['indice']==df_america.groupby('anio')['indice'].transform('min')]
minimo=df_america[df_america['indice']==df_america.groupby('anio')['indice'].transform('max')]
print("El pais con mejor indice por año fue")
display(maximo)
print("El pais con peor indice por año fue")
display(minimo)



El pais con mejor indice por año fue


,codigo_iso,anio,indice,ranking,pais
27,CAN,2001,0.80,2.0,Canadá
343,TTO,2002,1.00,2.0,Trinidad y Tobago
523,TTO,2003,2.00,30.0,Trinidad y Tobago
703,TTO,2004,2.00,31.0,Trinidad y Tobago
741,BOL,2005,4.50,63.0,Bolivia
747,CAN,2005,4.50,63.0,Canadá
927,CAN,2006,4.88,84.0,Canadá
1107,CAN,2007,3.33,50.0,Canadá
1287,CAN,2008,3.70,62.0,Canadá
1611,USA,2009,6.75,115.0,Estados Unidos


El pais con peor indice por año fue


,codigo_iso,anio,indice,ranking,pais
39,CUB,2001,90.30,99.0,Cuba
219,CUB,2002,97.83,125.0,Cuba
365,ARG,2003,35826.00,35.0,Argentina
579,CUB,2004,87.00,112.0,Cuba
759,CUB,2005,95.00,109.0,Cuba
939,CUB,2006,96.17,139.0,Cuba
1119,CUB,2007,88.33,117.0,Cuba
1299,CUB,2008,94.00,131.0,Cuba
1479,CUB,2009,78.00,129.0,Cuba
1659,CUB,2012,71.64,162.0,Cuba


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [ ]:
pivot_df=df_america.pivot_table(index='pais', columns='anio', values='indice', fill_value=0)
pivot_df

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Antigua y Barbuda,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.81,0.00,0.00,0.00,0.00,0.00
Argentina,12.0,15.17,35826.00,13.67,17.30,24.83,14.08,11.33,16.35,25.67,25.27,26.11,25.09,25.07,26.05,28.30
Belize,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,17.05,18.54,20.61,23.43,24.55,27.50
Bolivia,14.5,9.67,20.00,9.67,4.50,21.50,28.20,24.17,28.13,32.80,31.04,31.29,31.78,33.88,32.45,35.38
Brasil,18.8,16.75,16.50,14.50,17.17,25.25,18.00,15.88,16.60,32.75,34.03,31.93,32.62,33.58,31.20,32.79
Canadá,0.8,1.83,3.33,4.50,4.50,4.88,3.33,3.70,7.00,12.69,10.99,10.99,15.26,16.53,15.28,15.69
Chile,6.5,6.83,10.00,11.75,11.63,12.13,11.50,10.50,10.50,26.24,25.80,23.00,19.23,20.53,22.69,25.65
Colombia,40.8,49.17,47.38,40.17,44.75,42.33,35.50,40.13,51.50,37.48,36.68,39.08,44.11,41.47,41.03,42.82
Costa Rica,4.3,3.83,7.63,8.50,6.67,6.50,5.10,8.00,8.08,12.08,12.23,12.26,11.10,11.93,14.01,12.24


In [ ]:
# Calcular el valor máximo de toda la tabla dinámica
max_valor_tabla = pivot_df.max().max() # Calcula el maximo de cada columna, y luego el maximo de esos maximos

min_valor_tabla = pivot_df[pivot_df > 0].min().min() # Filtra los 0s y luego encuentra el minimo
ubicacion_max = pivot_df[pivot_df == max_valor_tabla].stack().index.tolist()
ubicacion_min = pivot_df[pivot_df == min_valor_tabla].stack().index.tolist()
print(f"El valor máximo ({max_valor_tabla}) se encuentra en:", ubicacion_max)
print(f"El valor mínimo (distinto de 0) ({min_valor_tabla}) se encuentra en:", ubicacion_min)


El valor máximo (35826.0) se encuentra en: [('Argentina', 2003)]
El valor mínimo (distinto de 0) (0.8) se encuentra en: [('Canadá', 2001)]


In [ ]:
# Calcular el promedio del indice para cada año (promedio de las columnas)
promedio_por_año = pivot_df.mean(axis=0)
promedio_maximo = promedio_por_año.idxmax()
promedio_minimo = promedio_por_año.idxmin()
print("El año con mayor indice promedio es:", promedio_maximo, "con un promedio de:", promedio_por_año.max())
print("El año con menor indice promedio es:", promedio_minimo,"con un promedio de:", promedio_por_año.min())

El año con mayor indice promedio es: 2003 con un promedio de: 1251.6758620689654
El año con menor indice promedio es: 2001 con un promedio de: 12.420689655172414


In [ ]:
variabilidad_por_pais = pivot_df.max(axis=1) - pivot_df.min(axis=1)
pais_max_variabilidad = variabilidad_por_pais.idxmax()
print("El pais con mayor variabilidad es:", pais_max_variabilidad)

El pais con mayor variabilidad es: Argentina


In [ ]:
min_por_pais = pivot_df.min(axis=1)
max_por_pais = pivot_df.max(axis=1)
min_por_pais==max_por_pais

,0
pais,
Antigua y Barbuda,False
Argentina,False
Belize,False
Bolivia,False
Brasil,False
Canadá,False
Chile,False
Colombia,False
Costa Rica,False


In [ ]:
#no existen paises con indices cosntantes

In [ ]:
no_data = pivot_df[pivot_df.eq(0).all(axis=1)]
paises_sin_datos = no_data.index.tolist()
print("Países sin datos:", paises_sin_datos)

Países sin datos: []
